# Arbyte Data Engineering

## Exploration

Import libraries

In [3]:
import pandas as pd
import huggingface_hub as hf_hub

Log into HuggingFace for access to data

In [4]:
hf_hub.notebook_login()

Import Dataset from AzharAli05/Resume-Screening-Dataset

In [5]:
# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_csv("hf://datasets/AzharAli05/Resume-Screening-Dataset/dataset.csv")

Extract Small Sample Size for Proof of Concept

In [33]:
poc_sample = df.head(20)
poc_sample

,Role,Resume,Decision,Reason_for_decision,Job_Description
0,E-commerce Specialist,Here's a professional resume for Jason Jones:\...,reject,Lacked leadership skills for a senior position.,Be part of a passionate team at the forefront ...
1,Game Developer,Here's a professional resume for Ann Marshall:...,select,Strong technical skills in AI and ML.,Help us build the next-generation products as ...
2,Human Resources Specialist,Here's a professional resume for Patrick Mccla...,reject,Insufficient system design expertise for senio...,We need a Human Resources Specialist to enhanc...
3,E-commerce Specialist,Here's a professional resume for Patricia Gray...,select,Impressive leadership and communication abilit...,Be part of a passionate team at the forefront ...
4,E-commerce Specialist,Here's a professional resume for Amanda Gross:...,reject,Lacked leadership skills for a senior position.,We are looking for an experienced E-commerce S...
5,Mobile App Developer,"Here's a sample resume for Jose Hall, a skille...",reject,No experience in back-end development.,We need a Mobile App Developer to enhance our ...
6,UX Designer,Here's a professional resume for Rachael Newma...,reject,Insufficient system design expertise for senio...,Help us build the next-generation products as ...
7,Cloud Engineer,"Here's a sample resume for Jessica Hall, a can...",reject,Needs improvement in machine learning algorithms.,We're seeking a talented Cloud Engineer to wor...
8,Digital Marketing Specialist,Here's a sample resume for Jonathan Powers:\n\...,select,Impressive leadership and communication abilit...,"As a Digital Marketing Specialist, you'll lead..."
9,AI Researcher,Here's a sample resume for Zachary Ward:\n\nZa...,select,Excellent full-stack development experience.,If you're passionate about software engineerin...


In [34]:
for resume in poc_sample["Resume"]:
    print(len(resume))

2613
429
3046
2344
2784
2978
3133
3362
3031
2774
2938
1513
2969
3022
2994
3181
2815
3352
2401
2840


In [35]:
for job_dec in poc_sample["Job_Description"]:
    print(len(job_dec))

137
110
136
136
129
129
107
100
93
129
125
94
166
99
100
99
129
134
82
132


## Filtering

In this section, we define our target labels as 'Decision', whether or not the application was accepted or rejected. Then we define the features for the ML model. For the PoC, we are not yet training the model, we will simulate the prediction of the model using the actual labels of the small sample size.